In [ ]:
# --------------------------------------------------
# Software_Realisation.ipynb
#
# Version 0.1
#
# Copyright (c) 2022 James Hinns
# This code is licensed under MIT license 
# (see LICENSE for details)
#
# Code to support the completion of my MSc.
#   Particularly supports the implementation
#   chapters 7 and 8.
#   Provides initial 'naive' implementations of 
#   the proposed underspecification index.
#   Follows with optimised implementations of the 
#   index, showing the performacne increases.
#
# Created 16/02/22 - adapted from previous work 
#                    alongside Dr. Fan Xiuyi
#
# James Hinns, Fan Xiuyi
# Swansea University
#--------------------------------------------------

# Setup

## Imports

In [7]:
import pandas as pd
import numpy as np

from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor

from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error

from sklearn.model_selection import train_test_split

from scipy.spatial.distance import cosine
from scipy.stats.stats import pearsonr
from scipy.stats import kendalltau

import shap

import os

from numba import njit, prange, typed

from numpy import dot
from numpy.linalg import norm

from timeit import default_timer as timer

## Read in test dataset

In [8]:
# Read in abalone dataset, split into train and test sets
# split train and test sets into training features x and
# prediction target y

df = pd.read_csv("abalone.csv")

df["f1"] = df["f1"].astype('category').cat.codes

# Create test 80% train 20% test sets
train, test = train_test_split(df, test_size=0.2)

x_trn = train.drop(columns=["label"])
y_trn = train['label']

x_tst = test.drop(columns=["label"])
y_tst = test['label']

# Naïve implementation

In [9]:
def gen_rashomon_naive(x_trn,y_trn,x_tst,y_tst,theta):
    """
    Naive implementation to generate Rashomon set R(theta)
    for the problem defined by the provided datasets
    using random forests.

            Parameters:
                x_trn - np.ndarray (2d-array): 
                    The numerical training features
                    used to train the predictors.
                    Each index should be all features 
                    for a training instance.
                y_trn - np.ndarray (2d-array):
                    The numerical training prediction
                    targets, each index should be the
                    prediction target for the features
                    at the same index of x_trn.
                x_tst - np.ndarray (2d-array):
                    Testing equivalent of x_trn.
                y_tst - np.ndarray (2d-array):
                    Testing equivalent of y_trn.

            Returns:
                rashomon - np.ndarray (1d-array):
                    Array of sklearn Random Forest models.
                    Length of MODEL_COUNT
    """
    
    
    MODEL_COUNT = 100
    TREE_COUNT  = 10

    rashomon = np.empty(MODEL_COUNT, dtype=object)
    i = 0
    
    while i < MODEL_COUNT:
        
        f = RandomForestClassifier(n_estimators=TREE_COUNT)
        
        f.fit(x_trn,y_trn)
        
        pred = f.predict(x_tst)
        
        acc = accuracy_score(pred, y_tst)
        
        if acc >= theta:
            rashomon[i] = f
            i += 1
            
    return rashomon

In [10]:
# For each data instance x in x_tst, 
#    Compute shap explanations for each predictor f in rashomon
def gen_exp_matrix_naive(x_tst,rashomon):
    """
    Naive implementation to generate an explanation matrix
    from a given Rashomon set and testing set.

            Parameters:
                x_tst - np.ndarray (2d-array):
                    The numerical testing features
                    used to generate the explanations.
                    Each index should be all features 
                    for a test instance.
                rashomon - np.ndarray (1d-array):
                    Array of sklearn Random Forest models.

            Returns:
                shap_exps - np.ndarray (3d-array),
                shape(x_tst length,model_count,# of features):
                    Array of SHAP explanations for each test instance
                    in x_tst, for each predictor in rashomon.
    """
    
    shap_exps = np.empty((len(x_tst)),dtype=object)
    
    for i, x in enumerate(x_tst):
        
        # An explation for each predictor, is a list of length equal to feature count
        local_exp = np.empty((len(rashomon)),dtype=object)
                         
        k = len(rashomon)
                             
        for j, f in enumerate(rashomon):
            
            # Ise tree explainer to give fast exact SHAP calc
            shap_val = np.array(shap.TreeExplainer(f).shap_values(x)[0])
            local_exp[j] = shap_val
            
        shap_exps[i] = local_exp
            
    return shap_exps

In [11]:
def calc_local_underspec_index_naive(local_exps,metrics):
    """
    Naive implementation to calculate the local underspecification
    index using selected metrics, given all the explanations for 
    a particular data instance for all predictors in a rashomon set

            Parameters:
                local_exps - np.ndarray (2d-array):
                    List of the local explanations for a given data 
                    instance of all predictors in a rashomon set.
                metrics - list:
                    List of metrics to be calculated.

            Returns:
                results - list:
                    The underspecification indexes using all metrics in
                    metrics.
    """
    # Output array, index for each metrics local index
    metric_n = len(metrics)
    results  = np.zeros((metric_n))
    
    n = len(local_exps)
    # Inner trianglular matrix coefficent for mean
    coef = ( 2 / (n*(n-1)) )
     
    # Inner trinaglular matrix pairwise calculate metrics of local_exps
    for i in range(n):
        
        # First predictor of pairwise comparison
        f1_e = local_exps[i]
        
        for j in range(i+1,n):
            
            # Second predictor of pairwise comparison
            f2_e = local_exps[j]
        
            # Calculate all requested metrics
            for m in range(metric_n):
                
                # Cosine Similarity
                if   ( metrics[m] == "c_sim" ):
                    metric = 1 - cosine(f1_e, f2_e)
                
                # Euclidean Distance
                elif ( metrics[m] == "e_dis" ):
                    metric = np.linalg.norm(f1_e-f2_e)
                
                # Pearson Correlation
                elif ( metrics[m] == "p_cor" ):
                    metric = pearsonr(f1_e,f2_e)[0]
                    
                elif ( metrics[m] == "k_tau" ):
                    metric = kendalltau(f1_e,f2_e)[0]
                    
                results[m] += coef * metric
                    
    return results

In [12]:
# Give an average underspceficiation index for a set of explanations
def set_underspecification_index_naive(exps,metrics):
    """
    Naive implementation to calculate the underspecifcation index
    for a given set of data instances.

            Parameters:
                exps - np.ndarray (3d-array),
                shape(x_tst length,model_count,# of features):
                    Array of explanations for a set of test instances,
                    for a number of predictors.
                metrics - list:
                    List of metrics to be calculated.

            Returns:
                set_i - np.ndarray (1d-array):
                    The set underspecification indexes using 
                    all metrics in metrics.
                local_is - np.ndarray (2d-array):
                    The local underspecification index for each 
                    metric in metrics, for each data instnace in
                    exps.
    """
    
    
    pred_n = len(exps)
    local_is = np.empty((pred_n),dtype=object)
    
    for i in range(pred_n):
        
        # Verbose printing
        if (i % 10 == 0):
            print(f"Calculating index {i}/{pred_n}")
        
        local_is[i] = calc_local_underspec_index_naive(exps[i],metrics)
    
    return local_is

## Naive Implementation Timing

In [17]:
# Subset example
s_x_tst = x_tst.to_numpy()[:100]
s_y_tst = y_tst.to_numpy()[:100]

s_x_trn = x_trn.to_numpy()
s_y_trn = y_trn.to_numpy()

metrics = ["e_dis","c_sim","p_cor","k_tau"]

# Time how long the whole operation takes
start = timer()

# Generate Rashomon set where all predictors f in R(theta), acc(f) >= theta
subset_rashomon = gen_rashomon_naive(s_x_trn,s_y_trn,
                                     s_x_tst,s_y_tst,
                                     0.7)

# Compute the explanations for all f in subset_rashomon, for each data instance x in s_exps
subset_exps     = gen_exp_matrix_naive(s_x_tst,
                                       subset_rashomon)

# Calculate the underspecification index for all data instance x in s_exps
local_is = set_underspecification_index_naive(subset_exps,
                                                     metrics)

end = timer()
print(end - start)

np.mean(local_is,axis=0)

Calculating index 0/100
Calculating index 10/100
Calculating index 20/100
Calculating index 30/100
Calculating index 40/100
Calculating index 50/100
Calculating index 60/100
Calculating index 70/100
Calculating index 80/100
Calculating index 90/100
255.4291273


array([0.12043251, 0.77302224, 0.72971872, 0.53260519])

# Optimisation

## Kendall Tau Implementation

In [18]:
@njit(fastmath=True)
def kendalltau_numba_naive(a,b):
    """
    Numba optimised o(n^2) kendall correlation coefficent 
    (tau-a) calculation implementation

            Parameters:
                 a,b - np.ndarray (1d.array):
                     Input arrays which will have their kendall
                     rank correlation calculated

            Returns:
                tau - float:
                    The Kendall correlation coefficent
    """
    
    a = np.argsort(a)
    b = np.argsort(b)
    
    n = len(a)
    
    # Calculate symettric difference
    sym_dis = 0
    
    for i in range(1,n):
        for j in range(0,i):
            sym_dis += np.sign(a[i] - a[j]) * np.sign(b[i] - b[j])
            
    tau = (2 * sym_dis) / (n*(n-1))   
        
    return tau

In [23]:
a = np.array([1,2,3])

kendalltau_numba_naive(a,a)

1.0

### Graph Creation

In [24]:
# Quick method to compare runtime of numba o(n^2) 
# to scipy is o(nlog) kendall tau implementations
def kendall_compare(iterations,list_length):
    
    a = np.random.rand(list_length)
    b = np.random.rand(list_length)
    
    start = timer()
    for i in range(iterations):
        kendalltau_numba_naive(a,b)
    end = timer()
    naive_t = end - start

    start = timer()
    for i in range(iterations):
        kendalltau(a.argsort(),b.argsort())
    end = timer()
    scipy_t = end - start
    
    return naive_t,scipy_t

In [72]:
list_lengths = np.concatenate(
    [np.arange(10,510,step=10),
     np.arange(1000,10500,step=500)]
)

iterations = 10000

naive_times,scipy_times = [],[]

for length in list_lengths:
    n,s = kendall_compare(iterations,length)
    naive_times += [n]
    scipy_times += [s]

In [83]:
# First run compiles, so we do the first comparison
n_10, s_10 = kendall_compare(10000,10)
naive_times[0] = n_10
scipy_times[0] = s_10

# Create dataframe for graph
n_t = ["naive" for i in range(len(list_lengths))]
s_t = ["SciPy" for i in range(len(list_lengths))]

methods = n_t + s_t
times   = naive_times + scipy_times
lenghts = np.concatenate([list_lengths,list_lengths])

df = pd.DataFrame({"Method":methods,"Time":times,"Length":lenghts})
df

,Method,Time,Length
0,naive,0.021183,10
1,naive,0.025686,20
2,naive,0.031423,30
3,naive,0.032620,40
4,naive,0.039078,50
...,...,...,...
133,SciPy,27.669746,8000
134,SciPy,28.629952,8500
135,SciPy,30.514546,9000
136,SciPy,32.008701,9500


In [84]:
import altair as alt

alt.Chart(df).mark_line().encode(
    x     = alt.X("Length:Q",
                  axis=alt.Axis(title="Input Length"),
                  scale=alt.Scale(type='log')
                 ),
    y     = alt.Y("Time:Q",
                  axis=alt.Axis(title="Runtime (s)"),
                  scale=alt.Scale(type='log')
                 ),
    color = "Method"
).properties(
    width=1920,
    height=1080
).configure_axis(
    labelFontSize=20,
    titleFontSize=25
).configure_legend(
    padding=10,
    strokeColor="gray",
    fillColor='#EEEEEE',
    orient='bottom-right',
    labelFontSize=20,
    titleFontSize=25,
)

alt.Chart(...)

## Numba NumPy metrics vs SciPy

In [64]:
# Metric njit functions
@njit(fastmath=True)
def p_r(a,b):
    np.corrcoef(a,b)[1,0]

@njit(fastmath=True)
def c_sim(a,b):
    np.dot(a, b)/(norm(a)*norm(b))

In [71]:
list_length = 1000
iterations  = 10000
    
a = np.random.rand(list_length)
b = np.random.rand(list_length)

start = timer()
for i in range(iterations):
    p_r(a,b)
end = timer()
print(f"Optimised Pearson: {end - start}")

start = timer()
for i in range(iterations):
    pearsonr(a,b)[0]
end = timer()
n_euc = end - start
print(f"SciPy Pearson: {n_euc}")

print()

start = timer()
for i in range(iterations):
    p_r(a,b)
end = timer()
print(f"Optimised Cosine: {end - start}")

start = timer()
for i in range(iterations):
    1 - cosine(a, b)
end = timer()
print(f"SciPy Cosine: {end - start}")

Optimised Pearson: 0.15633269999943877
SciPy Pearson: 0.6277061999999205

Optimised Cosine: 0.15588750000006257
SciPy Cosine: 0.3506908999997904


## Explanation Matrix Generation

In [25]:
# Compute shap explanations for each data instance x in x_tst, for model_count predictors f
# such that acc(f) >= theta
def gen_exp_matrix(x_trn,y_trn,x_tst,y_tst,theta,model_count=100,tree_count=10):
    """
    Generates an explanation matrix of the emprical Rashomon set defined by
    the given datasets, theta, model_count and tree_count.

            Parameters:
                x_trn - np.ndarray (2d-array): 
                    The numerical training features
                    used to train the predictors.
                    Each index should be all features 
                    for a training instance.
                y_trn - np.ndarray (2d-array):
                    The numerical training prediction
                    targets, each index should be the
                    prediction target for the features
                    at the same index of x_trn.
                x_tst - np.ndarray (2d-array):
                    Testing equivalent of x_trn.
                y_tst - np.ndarray (2d-array):
                    Testing equivalent of y_trn.
                model_count - int, default 100:
                    The number of models to include
                    in the Rashomon set. 
                tree_count - int, default 10:
                    The number of trees in each
                    random forest in the Rashomon set

            Returns:
                shap_exps - np.ndarray (3d-array),
                shape(model_count,x_tst length,# of features):
                    Array of SHAP explanations for each test instance
                    in x_tst, for each predictor in the Rashomon set.
    """
    
    # Throw an exception if datasets aren't numpy arrays
    if not all(isinstance(i, np.ndarray) for i in [x_trn,y_trn,x_tst,y_tst]):
        raise TypeError("Datasets must be NumPy arrays")

    # Model counter
    i = 0
    
    # Local explanation for each datapoint for each predictor
    out_shape = (model_count,len(x_tst),len(x_tst[0]))
    shap_exps = np.zeros(out_shape)
    
    while i < model_count:
        
        # Train model, calculate accuracy
        f = RandomForestClassifier(n_estimators=tree_count)
    
        f.fit(x_trn,y_trn)
        
        pred = f.predict(x_tst)
        
        acc = accuracy_score(pred, y_tst)
        
        # If f in R(theta)
        if acc >= theta:
            
            # Compute local shap exp for each x in x_tst
            shap_exps[i] = np.array(shap.TreeExplainer(f).shap_values(x_tst)[0])
            
            i += 1
            
    # Transpose output so that contiguous arrays can be passed to local underspec index
    return np.transpose(shap_exps,(1,0,2))

### Naive vs Optimised

In [26]:
# Time Naive exp_matrix
start = timer()

rashomon_naive = gen_rashomon_naive(s_x_trn,s_y_trn,
                                     s_x_tst,s_y_tst,
                                     0.7)

exps_naive = gen_exp_matrix_naive(s_x_tst,rashomon_naive)

end = timer()
print(end - start)

# Time updated version
start = timer()

exps = gen_exp_matrix(s_x_trn,s_y_trn,
                      s_x_tst,s_y_tst,
                      0.7,model_count=100)

end = timer()
print(end - start)

33.978633900000204
21.495222300000023


## Local Index U<sub>x</sub>

In [27]:
@njit(fastmath=True)
def calc_local_underspec_index(local_exps,metrics):
    
    """
    Calculates the local underspecification index using selected metrics,
    given all the explanations for a particular data instance for all 
    predictors in a Rashomon set

            Parameters:
                local_exps - np.ndarray (2d-array):
                    List of the local explanations for a given data 
                    instance of all predictors in a rashomon set.
                metrics - numba.typed.List:
                    List of metrics to be calculated.

            Returns:
                results - list:
                    The underspecification indexes using all metrics in
                    metrics.
    """
    
    # Output array, index for each metrics local index
    metric_n = len(metrics)
    results  = np.zeros((metric_n))
    
    n = len(local_exps)
    # Inner trianglular matrix coefficent for mean
    coef = ( 2 / (n*(n-1)) )
     
    # Inner trinaglular matrix pairwise calculate metrics of local_exps
    for i in range(n):
        
        # First predictor of pairwise comparison
        f1_e = local_exps[i]
        
        for j in range(i+1,n):
            
            # Second predictor of pairwise comparison
            f2_e = local_exps[j]
        
            # Calculate all requested metrics
            for m in range(metric_n):
                
                # Cosine Similarity
                if   ( metrics[m] == "c_sim" ):
                    metric = np.dot(f1_e, f2_e)/(norm(f1_e)*norm(f2_e))
                
                # Euclidean Distance
                if ( metrics[m] == "e_dis" ):
                    metric = norm(f1_e-f2_e)
                
                # Pearson Correlation
                elif ( metrics[m] == "p_cor" ):
                    metric = np.corrcoef(f1_e,f2_e)[1,0]
                
                # Kendall Correlation
                elif ( metrics[m] == "k_tau" ):
                    metric = kendalltau_numba_naive(f1_e,f2_e)
                
                results[m] += coef * metric
                    
    return results

### Naive vs Optimised

In [91]:
x_ind = 1

local_exps = exps[x_ind]

metrics = typed.List(["e_dis","c_sim","p_cor","k_tau"])

start = timer()
calc_local_underspec_index_naive(local_exps,metrics)
end = timer()
print(end - start)

start = timer()
calc_local_underspec_index(local_exps,metrics)
end = timer()
print(end - start)

2.4405737000015506
0.02964109999993525


## Set index U

In [86]:
# Give an average underspceficiation index for 
# a set of explanations
@njit
def set_underspecification_index(exps,metrics):
    
    pred_n = len(exps)
    local_is = np.zeros((pred_n,len(metrics)))
    
    for i in range(pred_n):
        
        # Verbose printing
        if (i % 10 == 0):
            print("Calculating index",i,"/",pred_n)
        
        local_is[i] = calc_local_underspec_index(exps[i],metrics)
    
    return local_is

In [87]:
# Removed verbose printing as it likely won't be in 
# order due to parallelisation
@njit(parallel=True)
def set_underspecification_index_parallel(exps,metrics):
    """
    Calculate the underspecifcation indexes
    for a given set of data instances.

            Parameters:
                exps - np.ndarray (3d-array),
                shape(x_tst length,model_count,# of features):
                    Array of explanations for a set of test instances,
                    for a number of predictors.
                metrics - list:
                    List of metrics to be calculated.

            Returns:
                local_is - np.ndarray (2d-array):
                    The local underspecification index for each 
                    metric in metrics, for each data instnace in
                    exps.
    """
    pred_n = len(exps)
    local_is = np.zeros((pred_n,len(metrics)))
    
    for i in prange(pred_n):
        
        local_is[i] = calc_local_underspec_index(exps[i],metrics)
    
    return local_is

In [92]:
start = timer()
set_underspecification_index_naive(exps,metrics)
end = timer()
print(end - start)

start = timer()
set_underspecification_index(exps,metrics)
end = timer()
print(end - start) 

start = timer()
set_underspecification_index_parallel(exps,metrics)
end = timer()
print(end - start)

Calculating index 0/100
Calculating index 10/100
Calculating index 20/100
Calculating index 30/100
Calculating index 40/100
Calculating index 50/100
Calculating index 60/100
Calculating index 70/100
Calculating index 80/100
Calculating index 90/100
243.66168079999989
Calculating index 0 / 100
Calculating index 10 / 100
Calculating index 20 / 100
Calculating index 30 / 100
Calculating index 40 / 100
Calculating index 50 / 100
Calculating index 60 / 100
Calculating index 70 / 100
Calculating index 80 / 100
Calculating index 90 / 100
2.8854897000001074
2.0547052999991138
